# Setup Inicial

## Bibliotecas utilizadas

In [5]:
import pandas as pd
import json

## Definições gerais

In [6]:
# Define os diretórios


blob_container_name = 'general' # replace with your container name
blob_relative_path_raw = 'raw/mercado_potencial/senatran/' # replace with your relative folder path
blob_relative_path_enriched = 'enriched/mercado_potencial/senatran/' # replace with your relative folder path
linked_service_raw = 'LS_ADLS_RAW_01' # replace with your linked service name
linked_service_enriched = 'LS_ADLS_ENRICHED_01' # replace with your linked service name


ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

#coletando o endpoint
end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]

#Utilizado na leitura via metodo mssparkutils
abfss_path_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
abfss_path_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)

dir_c = abfss_path_raw + '/condutores_habilitados/'
dir_f = abfss_path_raw + '/frota_veiculos/'


## Funções de utilizadas

In [7]:
# agrupa todos os arquivos .csv
def agrupa_df (lista, sep=',',dec='.',enc='utf-16'):
    """Agrupa dos dataframes.

        Lê os arquivos .csv e agrupa automaticamente os dataframes de dados do SENATRAN. 
        
    Parâmetros
    ----------
    lista: lista de strings 
        Lista com os endereços dos arquivos a serem lidos
    sep: string
        Separador dos dados conforme função pd.read_csv.
        padrão = ','
    dec: string
        simbolo de decimal conforme função pd.read_csv.
        padrão = '.'
    enc: string
        encolde de dados
        padrão = 'utf-16'
    
    Retorno
    -------
    df: pd.DataFrame
        Dataframe com os dados agrupados.
    """

    df = pd.DataFrame()
    for i in lista:
        x = pd.read_csv(i, sep=sep, decimal=dec, encoding=enc)
        x['COMPETENCIA'] = i.split('_')[-2] +'-'+i.split('_')[-1][:-4]+'-01'
        df = pd.concat([df,x], ignore_index=True)
    return(df)

# Execução

## Leitura e ajuste dos dados na camada row

In [8]:
# cria lista de arquivos .csv
lst_condutores = [i.path for i in mssparkutils.fs.ls(dir_c)]
lst_frota      = [i.path for i in mssparkutils.fs.ls(dir_f)]

# cria filtro para remover anos incompletos
remove = [str(x) for x in range(2011,2018)]
mes_dif = ['2021_12','2022_01'] # tabelas em formato diferente

condutores = [i for i in lst_condutores if all(key not in i for key in remove)]

# Frota tá uma salada os mes_dif estão em formato flat
frota      = [i for i in lst_frota if all(key not in i for key in remove)]
frota      = [i for i in lst_frota if all(key not in i for key in mes_dif)]
frota_     = [i for i in lst_frota if any(key in i for key in mes_dif)] # meses de frota com formato diferente

In [9]:
df_condut = agrupa_df(condutores, sep=',',dec='.',enc='utf-16')
df_frota  = agrupa_df(frota, sep=';',dec=',',enc='utf-8')
df_frota_  = agrupa_df(frota_, sep=';',dec=',',enc='utf-8') # dado em formato diferente dos demais

## Ajustes dos dados sobre a frota de veívculos

In [10]:
# agrupa todas as informações de coluna tipo em unica columa 
x = df_frota.columns.str.contains('TIPO')
df_frota['TP'] = df_frota[list(df_frota.columns[x])].bfill(axis=1).iloc[:, 0]
# remove as anteriores
df_frota.drop(df_frota.filter(regex='TIPO').columns, inplace=True, axis=1)

# reordena as colunas
order = [0,1,4,2,3]
df_frota = df_frota[df_frota.columns[order].to_list()]

# renomeia coluna TP para TIPO VEICULO
df_frota.rename({'TP':'TIPO VEICULO'}, axis=1, inplace=True)


In [11]:
# Ajusta Frota dos dois meses em formato diferente
x = ['UF', 'MUNICIPIO','COMPETENCIA']
y = df_frota_.columns.difference(x).to_list()

df_frota_ = pd.melt(df_frota_, id_vars=['UF','MUNICIPIO','COMPETENCIA'], value_vars=y)

# reordena as colunas
order = [0,1,3,4,2]
df_frota_ = df_frota_[df_frota_.columns[order].to_list()]

# renomeia colunas
df_frota_.rename({'variable':'TIPO VEICULO', 'value':'QUANTIDADE'}, axis=1, inplace=True)

del(x,y,order)

In [12]:
# Agrupando os dois dataframes de frota

df_frota_f = pd.concat([df_frota,df_frota_], axis=0)
df_frota_f.sort_values(['UF', 'MUNICIPIO', 'COMPETENCIA','TIPO VEICULO'], 
                      ascending=[True, True, True, True], inplace=True)
df_frota_f.reset_index(drop=True, inplace=True)

## Salvando dados mensais SENATRAN na camada eriched

In [13]:
df_condut.to_parquet(abfss_path_enriched + 'senatran_condutores_habilitados.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})

In [14]:
df_frota_f.to_parquet(abfss_path_enriched + 'senatran_frota_veiculos.parquet', 
                     storage_options = {'linked_service':linked_service_enriched})